# OpenAI Function Calling In LangChain

In [1]:
import os
import openai as OpenAI

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [ ]:
class User:
    def __init__(self, name:str, age: int, email: str):
        self.name= name
        self.age= age
        self.email= email

respondent1= User(name="Joe",age=32, email="joe@gmail.com")
respondent1.name

In [ ]:
respondent2= User(name="Joe",age='ticker', email="joe@gmail.com")
respondent2.age

* Class doesnot provide data vlidation

In [ ]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

respondent3 = pUser(name="Jane", age=32, email="jane@gmail.com")
respondent3.age

Next tine will through error

In [ ]:
respondent3 = pUser(name="Jane", age="ticker", email="jane@gmail.com")
respondent3.age

In [6]:
class Class(BaseModel):
    students: List[pUser]

In [7]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [8]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [3]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code : str= Field(description="airport code to get weather for")

In [4]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [5]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

C:\Users\oadsa\AppData\Local\Temp\ipykernel_6620\1313296718.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [13]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [14]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next cell is expected to generate an error.</p>

In [15]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [ ]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [ ]:
convert_pydantic_to_openai_function(WeatherSearch2)

In [6]:
from langchain.chat_models import ChatOpenAI

In [7]:
model = ChatOpenAI()

C:\Users\oadsa\AppData\Local\Temp\ipykernel_6620\1603725051.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [8]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 70, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--e568eb7b-ccb4-4975-91b9-850118e65c1d-0')

In [9]:
model_with_function = model.bind(functions=[weather_function])

In [10]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--7aa5e889-a02b-419f-b25c-05494c084627-0')

## Forcing it to use a function

We can force the model to use a function

In [ ]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("what is the weather in sf?")

In [ ]:
model_with_forced_function.invoke("hi!")

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [11]:
from langchain.prompts import ChatPromptTemplate

In [12]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [13]:
model = ChatOpenAI()

model_with_function = model.bind(functions=[weather_function])

In [14]:
chain = prompt | model_with_function

In [15]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 75, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--af5bd126-298a-424e-9009-795517c8613f-0')

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [16]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code : str= Field(description="airport code to get weather for")

class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [17]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [18]:
model_with_functions = model.bind(functions=functions)

In [19]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 116, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--acc2664d-74c1-4b74-abdb-52060830ec37-0')

In [20]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 118, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--00e183a2-c061-48eb-bcff-305a30bd4f62-0')

In [21]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--508e72c8-dbb8-4025-ba77-6349fcbda630-0')